### Set Up

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API Key
from api_keys import g_key

### Store Part 1 Results into DataFrame

In [2]:
weather_file = "Outputs/weather_data.csv"
weather_data = pd.read_csv(weather_file)

weather_data.head()

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Country,Date
0,Qaanaaq,77.4840,-69.3632,42.87,69,100,13.02,GL,1660426667
1,Cootamundra,-34.6500,148.0333,43.11,95,100,7.36,AU,1660426967
2,Rikitea,-23.1203,-134.9692,72.91,79,49,22.21,PF,1660426829
3,São José da Coroa Grande,-8.8978,-35.1478,74.62,70,75,13.27,BR,1660426970
4,Tubualá,9.5167,-79.0333,81.37,78,100,1.05,PA,1660426972


### Humidity Heatmap

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store 'lat' and 'lon' into locations

locations= weather_data[["Latitude", "Longitude"]].astype(float)

#Store Humidity by itself
humidity = weather_data["Humidity"].astype(float)

In [5]:
#Creating Humidity Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating = False, max_intensity = 100, point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### New DataFrame with My Ideal Weather

In [6]:
#perfect temp is at least 65 no higer than 79
perfect_temp = weather_data.loc[(weather_data["Max Temperature (F)"]<=79) & (weather_data["Max Temperature (F)"]>=65)]

#perfect clouds is 30% or less
perfect_clouds = perfect_temp.loc[perfect_temp["Cloudiness"]<=30]

#perfect humidity 55%-65%
perfect_humidity = perfect_clouds.loc[(perfect_clouds["Humidity"]<=65) & (perfect_clouds["Humidity"]>=55)]

#perfect wind no more than 9 mph
perfect_wind= perfect_humidity.loc[perfect_humidity["Wind Speed (mph)"] <=9]

#rename final data frame
my_vacation_spots_df= perfect_wind
my_vacation_spots_df

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Country,Date
138,Sept-Îles,50.2001,-66.3821,72.12,62,20,5.75,CA,1660426934
238,Neiafu,-18.6500,-173.9833,77.68,61,20,5.75,TO,1660427336
263,Rentina,40.6574,23.6140,72.91,63,9,3.00,GR,1660427376
322,Kurmanayevka,52.5093,52.0675,65.93,64,0,6.46,RU,1660427467
350,Havre-St-Pierre,50.2334,-63.5986,71.96,56,0,5.75,CA,1660427515
387,Margate,51.3813,1.3862,73.11,63,2,3.00,GB,1660427478
456,Reconquista,-29.1500,-59.6500,68.16,64,0,8.55,AR,1660427689
459,Saint-Pierre,-21.3393,55.4781,66.90,65,0,6.91,RE,1660427693
478,Karonga,-9.9333,33.9333,68.50,62,1,3.91,MW,1660427724
520,Sol'-Iletsk,51.1631,54.9918,67.86,58,10,8.50,RU,1660427796


### Hotel Map

In [7]:
#creating new datafram
hotel_df = my_vacation_spots_df

hotel_df["Hotel Name"]= ""

C:\Users\17sar\AppData\Local\Temp\ipykernel_50380\3982793158.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]= ""


In [8]:
#create a base url and add some parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius": 5000, "types": "lodging", "key": g_key}

In [9]:
#start of printing log
print("Beginning Data Retrieval")
print("-------------------------------")

for index,row in hotel_df.iterrows():
    
    #pull lat and lon for search
    lat=row["Latitude"]
    lon=row["Longitude"]
    
    #add location to parameter list
    params["location"] = f"{lat},{lon}"
    
    #assemble url and make api request
    print(f'Retrieving Results for Index {index}: {row["City"]}.')
    
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    #if data is found
    try:
        print(f'Closest hotel is {results[0]["name"]}.')
        
        hotel_df.loc[index, 'Hotel Name']= results[0]['name']
    
    #if data is not found
    except:
        print ("Missing Data....Skipping.")
        hotel_df.loc[index, 'Hotel Name'] = np.nan
        
    print("-------------------------------")

#End of printing log
print("Data Retrieval Complete")
print("-------------------------------")

Beginning Data Retrieval
-------------------------------
Retrieving Results for Index 138: Sept-Îles.
Closest hotel is Comfort Inn.
-------------------------------
Retrieving Results for Index 238: Neiafu.
Closest hotel is Mystic Sands Beachfront Accommodation.
-------------------------------
Retrieving Results for Index 263: Rentina.
Closest hotel is Spring.
-------------------------------
Retrieving Results for Index 322: Kurmanayevka.
Missing Data....Skipping.
-------------------------------
Retrieving Results for Index 350: Havre-St-Pierre.
Closest hotel is Auberge Niapiskau.
-------------------------------
Retrieving Results for Index 387: Margate.
Closest hotel is Premier Inn Margate hotel.
-------------------------------
Retrieving Results for Index 456: Reconquista.
Closest hotel is Grand Hotel.
-------------------------------
Retrieving Results for Index 459: Saint-Pierre.
Closest hotel is Alize Plage.
-------------------------------
Retrieving Results for Index 478: Karonga.


In [10]:
hotel_data_clean= hotel_df.dropna()
hotel_data_clean

,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed (mph),Country,Date,Hotel Name
138,Sept-Îles,50.2001,-66.3821,72.12,62,20,5.75,CA,1660426934,Comfort Inn
238,Neiafu,-18.6500,-173.9833,77.68,61,20,5.75,TO,1660427336,Mystic Sands Beachfront Accommodation
263,Rentina,40.6574,23.6140,72.91,63,9,3.00,GR,1660427376,Spring
350,Havre-St-Pierre,50.2334,-63.5986,71.96,56,0,5.75,CA,1660427515,Auberge Niapiskau
387,Margate,51.3813,1.3862,73.11,63,2,3.00,GB,1660427478,Premier Inn Margate hotel
456,Reconquista,-29.1500,-59.6500,68.16,64,0,8.55,AR,1660427689,Grand Hotel
459,Saint-Pierre,-21.3393,55.4781,66.90,65,0,6.91,RE,1660427693,Alize Plage
478,Karonga,-9.9333,33.9333,68.50,62,1,3.91,MW,1660427724,Sumuka Inn
520,Sol'-Iletsk,51.1631,54.9918,67.86,58,10,8.50,RU,1660427796,Raduzhnaya


In [11]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_data_clean.iterrows()]
locations = hotel_data_clean[["Latitude", "Longitude"]]

#create markers
markers= gmaps.marker_layer(locations,info_box_content=hotel_info)


In [12]:
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))